In [14]:
import csv
import json

from merkle import *

In [15]:
with open('data/ownership.csv', 'r') as f:
    reader = csv.reader(f)
    index_tuples = list(reader)

Ok, let's merkle tree up this list of leaves with keccak256 goodness

Create a map of merkle leaf -> branch and store the root. Then serialize this entire object in json.

Ah-ha! There's a way to do this in web3py: https://web3py.readthedocs.io/en/stable/web3.main.html?highlight=solidityKeccak#web3.Web3.solidityKeccak

Merkle alg:
- store leaf -> [branch]
- store last_layer_node -> leaf
- until len(last_layer) == 1
  - hash pairs of last_layer to create new layer
  - add other side of pair + new parent to branches[leaf[last_layer_node]]
- return size 1 last_layer (root) + leaf -> [branch]

In [16]:
tree = build_tree(index_tuples)

Ok, now we should keep these somewhere handy. Next step is to *verify* each merkle branch. If any don't verify, note that somewhere.

Let's just make sure they're all consistent as a final sanity check on today's work.

Well.... shit. They don't match. It's the ordering thing! I need to have a standard ordering for the combination method. And this should be do-able within the contract as well!

Sure enough, there's a openzepplin lib I can use for this and that lib sorts before combining: https://github.com/OpenZeppelin/openzeppelin-contracts/blob/master/contracts/cryptography/MerkleProof.sol#L15-L32

In [17]:
leaf_to_branch = tree.leaf_to_branch
root = tree.root

In [18]:
for leaf, branch in leaf_to_branch.items():
    if not check_branch(root, leaf, branch):
        print("roots don't match!")

Success! Let's store the merkle data in json and prep it for sharing later.

In [19]:
with open('data/merkle_tree.json', 'w') as f:
    json.dump(tree.jsonify(), f)

Misc scratch work:

In [7]:
b = hash_leaf((1,'0x01465b5a079a742e66ef3e6184ef6d125eaa1de8201339cf32503c32ac1624e8','0x433A15f56e95Ee632dc690C032B5B2F7de447446',33))
b.hex()

b2 = hash_leaf((2912,'0xfdace08abdccfb2ab62a6f0fd964bc1e51419ae4df69f473f2ed2738d8280443','0xf7dBFe7dcFBA501464008554e7c5EddE8ab7B0ff',27))
b2.hex()

'c2fd78394ac23461309bcad6b514dfa259a7b5db18fc2b2b08d53ea1451d0491'